# VacationPy
----

#### Note
* Keep an eye on your API usage. Use https://developers.google.com/maps/reporting/gmp-reporting as reference for how to monitor your usage and billing.

* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [1]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os

# Import API key
from api_keys import g_key

### Store Part I results into DataFrame
* Load the csv exported in Part I to a DataFrame

In [2]:
weather_df = pd.read_csv("output_data/cities.csv")

weather_df.head()

,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,Darhan,49.4867,105.9228,53.10,79,99,12.21,MN,1632189785
1,New Norfolk,-42.7826,147.0587,49.50,58,99,9.19,AU,1632189785
2,Solano,38.2333,-121.9511,86.18,18,1,3.44,US,1632189786
3,Busselton,-33.6500,115.3333,61.66,53,1,12.06,AU,1632189786
4,Asău,46.4333,26.4000,45.52,91,96,0.60,RO,1632189787


### Humidity Heatmap
* Configure gmaps.
* Use the Lat and Lng as locations and Humidity as the weight.
* Add Heatmap layer to map.

In [3]:
# Configure gmaps
gmaps.configure(api_key=g_key)

# Store latitude and longitude in locations
locations = weather_df[["Lat", "Lng"]]

# Fill NaN values and convert to float
humidity = weather_df['Humidity'].astype(int)

In [4]:
# Plot Heatmap
>>> fig = gmaps.figure(center=(0, 0), zoom_level=2, layout={'padding': '3px','border': '1px solid black'
})

# Create heat layer
heat_layer = gmaps.heatmap_layer(locations, weights = humidity, 
                                 dissipating = False, max_intensity = 10,
                                 point_radius = 1)


# Add layer
fig.add_layer(heat_layer)

# Display figure
fig

Figure(layout=FigureLayout(border='1px solid black', height='420px', padding='3px'))

### Create new DataFrame fitting weather criteria
* Narrow down the cities to fit weather conditions.
* Drop any rows will null values.

In [5]:
# Narrowing the weather dataframe to include the following parameters:
    # A max temperature lower than 80 degrees but higher than 70.
    # Wind speed less than 10 mph.
    # Zero cloudiness.

perfect_weather_df = weather_df[(weather_df['Max Temp'] < 80) & (weather_df['Max Temp'] > 70) & (weather_df['Wind Speed'] < 10) & (weather_df['Cloudiness'] == 0)]
perfect_weather_df.head(40)

,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
74,Yinchuan,38.4681,106.2731,76.32,19,0,6.33,CN,1632189824
159,Riyadh,24.6877,46.7219,75.34,18,0,6.42,SA,1632189820
195,Zhoucheng,35.9122,116.3117,74.32,68,0,5.79,CN,1632189889
264,Dörtyol,36.8616,36.2289,71.26,78,0,2.55,TR,1632189930
275,Ankang,32.6800,109.0172,73.04,71,0,2.46,CN,1632189936
283,Qurayyat,31.3318,37.3428,73.92,43,0,5.70,SA,1632189940
365,Abu Samrah,35.3029,37.1841,73.24,46,0,5.19,SY,1632189986
376,Kiryat Gat,31.6100,34.7642,74.08,76,0,4.43,IL,1632189992
497,Natal,-5.7950,-35.2094,75.85,83,0,6.91,BR,1632190056
506,Lichuan,30.3000,108.8500,72.14,48,0,1.12,CN,1632190061


### Hotel Map
* Store into variable named `hotel_df`.
* Add a "Hotel Name" column to the DataFrame.
* Set parameters to search for hotels with 5000 meters.
* Hit the Google Places API for each city's coordinates.
* Store the first Hotel result into the DataFrame.
* Plot markers on top of the heatmap.

In [6]:
# Creating the new hotel frame by pulling the data of the cities meeting the weather parameters.
filter_city = perfect_weather_df['City'].tolist()
filter_lat = perfect_weather_df['Lat'].tolist()
filter_lng = perfect_weather_df['Lng'].tolist()
filter_country = perfect_weather_df['Country'].tolist()

hotel_dict = {'City': filter_city, 'Country': filter_country, 'Lat': filter_lat, 'Lng': filter_lng}
hotel_df = pd.DataFrame.from_dict(hotel_dict)

# Adding the new columns where the hotel information will go.
hotel_df['Hotel Name'] = ""
hotel_df["Hotel Address"] = ""
hotel_df["Hotel Rating"] = ""

In [7]:
# Params dictionary to update each iteration
params = {
    "radius": 5000,
    "keyword": "hotel",
    "key": g_key
}

# Pulling the latitude and longitude values from the filtered cities.
for index, row in hotel_df.iterrows():
    lat = row["Lat"]
    lng = row["Lng"]

    # Sets the location parameter to update each time.
    params["location"] = f"{lat},{lng}"

    # Setting the search terms: "Hotel" and our city location.
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"

    # Making the requests and converting to json
    name_address = requests.get(base_url, params=params).json()

    # Since some data may be missing we incorporate a try-except to skip any that are missing a data point.
    try:
        hotel_df.loc[index, "Hotel Name"] = name_address["results"][0]["name"]
        hotel_df.loc[index, "Hotel Address"] = name_address["results"][0]["vicinity"]
        hotel_df.loc[index, "Hotel Rating"] = name_address["results"][0]["rating"]
    except (KeyError, IndexError):
        print("Missing field/result... skipping.")

Missing field/result... skipping.
Missing field/result... skipping.


In [8]:
# Removing the cities we could not find hotel information for.
found_hotel_df = hotel_df[hotel_df['Hotel Name']!=""].reset_index()

found_hotel_df

,index,City,Country,Lat,Lng,Hotel Name,Hotel Address,Hotel Rating
0,0,Yinchuan,CN,38.4681,106.2731,Kempinski Hotel Yinchuan,"160 Beijing Middle Road, Yinchuan",4.7
1,1,Riyadh,SA,24.6877,46.7219,"Radisson Blu Hotel, Riyadh","King Abdulaziz Street, Al Mubarakiah Plaza, Ri...",4.1
2,3,Dörtyol,TR,36.8616,36.2289,Fourway Hotel,Sanayi Mahallesi Ali İhsan Bölükbaşı Bulvarı N...,4.1
3,4,Ankang,CN,32.6800,109.0172,Mingjiang International Hotel,"3 Binjiang Ave, Hanbin District, Ankang",4.7
4,5,Qurayyat,SA,31.3318,37.3428,Dana North Apartment,"Umar Ibn Al Khattab Rd, Al Qurayyat",4
5,7,Kiryat Gat,IL,31.6100,34.7642,Desert Gat,"Sderot HaAtsma'ut 134, Kiryat Gat",4.8
6,8,Natal,BR,-5.7950,-35.2094,Aram Imirá Plaza Hotel & Convention,"Via Costeira Sen. Dinarte Medeiros Mariz, 4077...",4.1
7,9,Lichuan,CN,30.3000,108.8500,Yicheng Hotel,"Zhutian Village, Lichuan",0
8,10,Polis,CY,35.0367,32.4264,Anassa Hotel,"40, Alekou Michailidi road Neo Chorio",4.4


In [28]:
# NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in found_hotel_df.iterrows()]
hotel_locations = found_hotel_df[["Lat", "Lng"]]

In [31]:
# Add marker layer ontop of heat map

# Storing the locations and names of the hotels that were found.
marker_locations = found_hotel_df[['Lat', 'Lng']]
hotel_names = found_hotel_df['Hotel Name']

# Defining and adding the marker overlay for each hotel.
markers = gmaps.marker_layer(marker_locations,
    info_box_content=[f"Nearby Hotel: {hotel}" for hotel in hotel_names])
fig.add_layer(markers)

# Display figure
fig


Figure(layout=FigureLayout(border='1px solid black', height='420px', padding='3px'))